## como usar
- colocar o arquivo csv do Cora na pasta: **2 - [FINANCEIRO]/[2023]/Balanço**
- o nome do arquivo sempre deve ser: **yyyy-mm.csv**
- na 2a celula, **trocar o nome** do arquivo csv acima 
- rodar todas as celulas **em ordem**

## fazer manualmente na planilha
- ajustar o tamanho das colunas
- **formatar como moeda** a entrada e saida
- (um dia arrumo tudo isso)

In [ ]:
from google.colab import drive
import os

import pandas as pd
import numpy as np
from openpyxl import load_workbook
!pip install xlsxwriter

drive.mount('/content/gdrive')
os.chdir('/content/gdrive/My Drive/2 - [FINANCEIRO]/[2023]/Balanço')

In [ ]:
# abre o arquivo csv
file = '2023-03.csv'
df = pd.read_csv(file, encoding='utf-8-sig', sep=',', dtype=str)

In [ ]:
def month_mapper(month):
     keys = {
         "01": "JANEIRO",
         "02": "FEVEREIRO",
         "03": "MARÇO",
         "04": "ABRIL",
         "05": "MAIO",
         "06": "JUNHO",
         "07": "JULHO",
         "08": "AGOSTO",
         "09": "SETEMBRO",
         "10": "OUTUBRO",
         "11": "NOVEMBRO",
         "12": "DEZEMBRO"
     }

     return keys[month]

def number_formatter(number):
    number = float(number)
    return "{:,.2f}".format(number).replace(',', '#').replace('.', ',').replace('#', '.')

In [ ]:
# formata os valores com separador de milesimo para . e de decimal para , 
df['Valor'] = df['Valor'].astype(float)
df['Valor'] = df['Valor'].apply(lambda x: number_formatter(x))

# Divide os valores para a coluna entrada e saida
df['Entrada R$'] = df.apply(lambda x: x['Valor'] if x['Tipo Transação'] == 'CRÉDITO' else '', axis=1)
df['Saida R$'] = df.apply(lambda x: x['Valor'] if x['Tipo Transação'] == 'DÉBITO' else '', axis=1)
df = df.drop(columns=['Valor', 'Tipo Transação'], axis=1)

# cria coluna de historico
df['Historico'] = ''

# cria coluna detalhes concatenando valores da transacao e identificacao
df['Detalhes'] = df[['Transação', 'Identificação']].apply(': '.join, axis=1)
df = df.drop(columns=['Transação', 'Identificação'], axis=1)

# formata df
df = df.reindex(columns=['Data', 'Historico', 'Detalhes', 'Entrada R$', 'Saida R$'])
df = df.sort_values(by = 'Data')

In [ ]:
month = month_mapper(file[5:7])

book = load_workbook('[BALANÇO CPE] [2023].xlsx')
writer = pd.ExcelWriter('[BALANÇO CPE] [2023].xlsx', engine='openpyxl', mode='a', if_sheet_exists='overlay')
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in writer.book.worksheets)

df.to_excel(writer, sheet_name=month, index=False, header=False, startcol=1, startrow=4)

'''
# engine'xlsxwriter'
for column in df:
    column_width = max(df[column].astype(str).map(len).max(), len(column))
    col_idx = df.columns.get_loc(column)
    writer.sheets['MARÇO'].set_column(col_idx, col_idx, column_width)
'''

writer.save()